In [1]:
import pandas as pd
import numpy as np
import os
from tqdm.notebook import tqdm
from convert_pmt_ids import *

In [2]:
import re

_nsre = re.compile('([0-9]+)')
def natural_sort_key(s):
    return [int(text) if text.isdigit() else text.lower()
            for text in re.split(_nsre, s)]   

In [3]:
PMTPos_CD_LPMT = pd.read_csv('PMTPos_CD_LPMT.csv')

In [4]:
for en_value in tqdm(range(9)):
    data_type = 'test/' #'train/'

    if data_type=='test/':
        energy = '/e+_{}/'.format(en_value)
    else:
        energy = ''

    path = 'data/real/'+data_type

    files_data = os.listdir(path+'data'+energy)
    files_data.sort(key=natural_sort_key)

    files_targets = os.listdir(path+'targets'+energy)
    files_targets.sort(key=natural_sort_key)

    thr_array = [1 + i for i in range(9)] + [5 * (i + 1) for i in range(1, 18)] + [91 + i for i in range(9)]

    df = pd.DataFrame()
    for k in tqdm(range(len(files_data)), leave=False):   
        data = np.load(path+'data/'+energy+files_data[k], allow_pickle=True)['a']

        NEvents = data.shape[1]
        for i in tqdm(range(NEvents), leave=False):
            data[0, :][i] = convert_pmt_ids(data[0, :][i])

        lpmt_charge = data[1, :]
        lpmt_fht_array = data[2, :]

        x_cc_array = np.zeros(NEvents)
        y_cc_array = np.zeros(NEvents)
        z_cc_array = np.zeros(NEvents)

        accum_charge = np.zeros(NEvents)

        for i in tqdm(range(NEvents), leave=False):
            lpmt_x = np.array(PMTPos_CD_LPMT['x'][data[0, :][i]]) / 1000.
            lpmt_y = np.array(PMTPos_CD_LPMT['y'][data[0, :][i]]) / 1000.
            lpmt_z = np.array(PMTPos_CD_LPMT['z'][data[0, :][i]]) / 1000.

            x_cc = np.sum(lpmt_x * lpmt_charge[i]) / np.sum(lpmt_charge[i])
            y_cc = np.sum(lpmt_y * lpmt_charge[i]) / np.sum(lpmt_charge[i])
            z_cc = np.sum(lpmt_z * lpmt_charge[i]) / np.sum(lpmt_charge[i])

            x_cc_array[i] = x_cc
            y_cc_array[i] = y_cc
            z_cc_array[i] = z_cc

            accum_charge[i] = np.sum(lpmt_charge[i])

        R_cc_array = (x_cc_array**2 + y_cc_array**2 + z_cc_array**2)**0.5
        pho_cc_array = (x_cc_array**2 + y_cc_array**2)**0.5

        ht_ps = []
        for thr in tqdm(thr_array, leave=False):
            ht_ps.append([np.percentile(data[2, :][i], thr) for i in range(NEvents)])

        features_df = pd.DataFrame()
        features_df['AccumCharge'] = accum_charge
        features_df['R_cc'] = R_cc_array
        features_df['pho_cc'] = pho_cc_array
        features_df['ht_2p'] = ht_ps[1]
        features_df['ht_20p'] = ht_ps[11]
        features_df['ht_35p'] = ht_ps[14]
        features_df['ht_75p'] = ht_ps[22]

        if data_type=='train/':

            targets = pd.read_csv(path+'targets/'+files_targets[k])
            targets['edepR'] = (targets['edepX']**2 + targets['edepY']**2 + targets['edepZ']**2)**0.5
            features_df = pd.concat([features_df, targets], axis=1)
        df = df.append(features_df)

    if data_type=='test/':
        targets = pd.read_csv(path+'targets/'+energy+files_targets[0])
        targets['edepR'] = (targets['edepX']**2 + targets['edepY']**2 + targets['edepZ']**2)**0.5
        df = df.reset_index().drop('index', axis=1)
        df = pd.concat([df, targets], axis=1)

    df.to_csv('ProcessedTestReal/{}MeV.csv.gz'.format(en_value),index=False, compression='gzip')

  0%|          | 0/9 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/35 [00:00<?, ?it/s]

...

  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/35 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/35 [00:00<?, ?it/s]